In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Thu Jul  1 15:31:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:3B:00.0 Off |                  N/A |
| 22%   29C    P0    33W / 250W |      0MiB / 12066MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN V             Off  | 00000000:5E:00.0 Off |                  N/A |
| 21%   

In [2]:
import os
os.chdir('/home/l/liny/ruofan/pytorch-metric-learning/src')
os.environ["CUDA_VISIBLE_DEVICES"]="1, 0"

In [3]:
from pytorch_metric_learning import losses, miners, distances, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from torchvision import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

from pytorch_metric_learning.models import bninception
from pytorch_metric_learning import samplers
from pytorch_metric_learning.datasets.Car196.data_loaders import *

/home/l/liny/anaconda3/envs/mypy37/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [4]:
device = torch.device("cuda")

train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

batch_size = 64
num_epochs = 200
num_classes = 196
hidden_dim = 512
centers_per_class = 5
result_dir = './log'
exp_name = 'Car196'
os.makedirs(result_dir, exist_ok=True)

In [5]:
train_data = CarsDataset(
                        data_dir='pytorch_metric_learning/datasets/Car196/datasets/training/extracted/', 
                        metas='pytorch_metric_learning/datasets/Car196/datasets/cars_metas/cars_train_annos.mat', 
                        transform=train_transform,
                            )

In [6]:
test_data = CarsDataset(
                        data_dir='pytorch_metric_learning/datasets/Car196/datasets/testing/extracted/', 
                        metas='pytorch_metric_learning/datasets/Car196/datasets/cars_metas/cars_test_annos_withlabels.mat', 
                        transform=test_transform,
                            )

In [7]:
sampler = samplers.MPerClassSampler(train_data.target, 
                                    m=5, 
                                    length_before_new_iter=100000)

In [8]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [9]:
for data, target in train_loader:
    print(target)
    break

tensor([156, 156, 156, 156, 156,  45,  45,  45,  45,  45,  16,  16,  16,  16,
         16,  15,  15,  15,  15,  15, 103, 103, 103, 103, 103, 110, 110, 110,
        110, 110,  17,  17,  17,  17,  17,   6,   6,   6,   6,   6,   9,   9,
          9,   9,   9,  21,  21,  21,  21,  21,  32,  32,  32,  32,  32,  50,
         50,  50,  50,  50, 107, 107, 107, 107])


In [10]:
model = bninception(dim=hidden_dim, pretrained=None)
model = torch.nn.DataParallel(model).to(device)

In [11]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ### 
def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print("Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(epoch, batch_idx, loss, mining_func.num_triplets))

### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(test_embeddings, 
                                                  train_embeddings,
                                                  test_labels,
                                                  train_labels,
                                                  False)
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))
    return accuracies["precision_at_1"]

In [12]:
### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.MeanReducer()
loss_func = losses.SoftTripleLoss(num_classes=num_classes, 
                                  embedding_size=hidden_dim, 
                                  centers_per_class=centers_per_class, 
                                  la=20, 
                                  gamma=0.1, 
                                  margin=0.01).to(device)

loss_optimizer = torch.optim.Adam([{"params": model.parameters(), "lr": 1e-4},
                                   {"params": loss_func.parameters(), "lr": 1e-2}])

mining_func = miners.TripletMarginMiner(margin = 0.2, distance = distance, type_of_triplets = "semi-hard")
accuracy_calculator = AccuracyCalculator(include = ("precision_at_1",), k = 1)

In [ ]:
### pytorch-metric-learning stuff ###

for epoch in range(1, num_epochs+1):
    train(model, loss_func, mining_func, device, train_loader, loss_optimizer, epoch)
    knn_acc = test(train_data, test_data, model, accuracy_calculator)
    if epoch % 50 == 0 or epoch == 1:
        torch.save(model.state_dict(), os.path.join(result_dir, '{}_epoch{}_knnAcc{:.4f}.pt'.format(exp_name, epoch, knn_acc)))


Epoch 1 Iteration 0: Loss = 5.238691329956055, Number of mined triplets = 14378
Epoch 1 Iteration 100: Loss = 5.0792341232299805, Number of mined triplets = 10792
Epoch 1 Iteration 200: Loss = 4.540863990783691, Number of mined triplets = 11724
Epoch 1 Iteration 300: Loss = 4.045415878295898, Number of mined triplets = 10405
Epoch 1 Iteration 400: Loss = 3.2982752323150635, Number of mined triplets = 9695
Epoch 1 Iteration 500: Loss = 4.1106157302856445, Number of mined triplets = 9902
Epoch 1 Iteration 600: Loss = 2.7585184574127197, Number of mined triplets = 8452
Epoch 1 Iteration 700: Loss = 2.9367265701293945, Number of mined triplets = 8601
Epoch 1 Iteration 800: Loss = 2.873548984527588, Number of mined triplets = 8519
Epoch 1 Iteration 900: Loss = 2.69873046875, Number of mined triplets = 6418
Epoch 1 Iteration 1000: Loss = 1.5383989810943604, Number of mined triplets = 5469
Epoch 1 Iteration 1100: Loss = 1.411964774131775, Number of mined triplets = 4992
Epoch 1 Iteration 1200

100%|██████████| 252/252 [00:19<00:00, 12.82it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.46412139711901546
Epoch 2 Iteration 0: Loss = 0.6029092073440552, Number of mined triplets = 4709
Epoch 2 Iteration 100: Loss = 0.6121525764465332, Number of mined triplets = 4493
Epoch 2 Iteration 200: Loss = 0.40242645144462585, Number of mined triplets = 3826
Epoch 2 Iteration 300: Loss = 0.5477001667022705, Number of mined triplets = 4915
Epoch 2 Iteration 400: Loss = 0.3680006265640259, Number of mined triplets = 5314
Epoch 2 Iteration 500: Loss = 0.39715105295181274, Number of mined triplets = 4742
Epoch 2 Iteration 600: Loss = 0.2457796186208725, Number of mined triplets = 3972
Epoch 2 Iteration 700: Loss = 0.3505677580833435, Number of mined triplets = 4123
Epoch 2 Iteration 800: Loss = 0.19687652587890625, Number of mined triplets = 5007
Epoch 2 Iteration 900: Loss = 0.22015593945980072, Number of mined triplets = 2672
Epoch 2 Iteration 1000: Loss = 0.23871999979019165, Number of mined triplets = 4215
Epoch 2 Iteration 110